# Regression Model

## Introduction
In today's modern age, technological devices have become an integral part of society. With ever-evolving computer components, it can be difficult to conduct research on a laptop that fits an individual's specific needs. In our algorithm, we will be providing price estimates based on the hardware specifications provided in our dataset. For instance, we could use variables such as CPU speed and RAM size to predict the price of a laptop. Ultimately, our goal is to assist the users and companies by providing a price estimate for their ideal laptop, thus, reducing the time needed for research. Thus our predictive question would be **"What will be the price of a laptop based on its specifications?"**. The dataset we will be using is an open-source file from Kaggle. Link for the original dataset: https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset/data'

## Preparing Libraries and Setup

Unfortunately, the python language does not natively support everything we will use to create this model, and it would be inefficient to re-invent the wheel. As such, we will be using the following libraries in our code:
#### **Pandas**
##### We use the DataFrame object from pandas to store and manipulate our data

#### **Altair**
##### Visualization library that allows us to make charts to help see the data

#### **Numpy**
##### Math library. We use it to set seeds and convert number Types. Also used by pandas in many operations

#### **Scikit learn**
##### Machine learning library. This will be the backbone of our regression model. We use it to build and train the model on our data, and to test the results.
 
#### **Matplotlib**
##### (insert something here)



In [1]:
### Uncomment cell below whenever Altair stops working to reinstall latest version

## For some reason, whenever the jupyter server restarts, it
## sends you back to the old version of altair (4.2.2)

In [2]:
#pip install -U altair      #<---- UNCOMMENT

In [3]:
## If the text below says anything below version 5.0.0,
## run the code above
import altair as alt; alt.__version__

'5.2.0'

In [4]:
### Run this cell before continuing.

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")
    
np.random.seed(1137110237) #Randomly picked seed

### Importing and Wrangling Data

(INSERT DESCRIPTION)
<br> Link for the original dataset: https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset/data'

In [5]:
# Loading csv file data as a pandas dataframe
laptop_data = pd.read_csv("https://raw.githubusercontent.com/fyip3/ds_project/main/data/laptopData.csv")

# Cleaning data
laptop_data = laptop_data.drop(columns=["Unnamed: 0"])          # Filtering Columns
laptop_data = laptop_data.dropna()                              # Removing redundant non-numeric part
laptop_data['Ram'] = laptop_data['Ram'].str.extract('(\d+)', expand=False)
laptop_data['Weight'] = laptop_data['Weight'].str.removesuffix("kg")
laptop_data['Memory'] = laptop_data['Memory'].str.extract('(\d+)', expand=False)
laptop_data["Price"] = laptop_data["Price"] * 0.017                         # Convert Price from INR to CAD
laptop_data = laptop_data.rename(columns={"Inches": "ScreenSize_Inches", "Ram": "Memory_GB", "Memory" : "Storage", "Weight" : "Weight_Kg", "Price" : "Price_CAD"})
# Convert columns from strings to int/float
laptop_data["Memory_GB"] = pd.to_numeric(laptop_data.Memory_GB, errors='coerce')
laptop_data["Weight_Kg"] = pd.to_numeric(laptop_data.Weight_Kg, errors='coerce')
laptop_data["ScreenSize_Inches"] = pd.to_numeric(laptop_data.ScreenSize_Inches, errors='coerce')
laptop_data["Storage"] = pd.to_numeric(laptop_data.Storage, errors='coerce')
laptop_data.dtypes
count = laptop_data.nunique()  #number of unique datapoints for each varaible
count
laptop_data = laptop_data.dropna()

In [6]:
laptop_data

,Company,TypeName,ScreenSize_Inches,ScreenResolution,Cpu,Memory_GB,Storage,Gpu,OpSys,Weight_Kg,Price_CAD
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128.0,Intel Iris Plus Graphics 640,macOS,1.37,1213.437614
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128.0,Intel HD Graphics 6000,macOS,1.34,814.223894
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256.0,Intel HD Graphics 620,No OS,1.86,520.812000
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512.0,AMD Radeon Pro 455,macOS,1.83,2298.320712
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256.0,Intel Iris Plus Graphics 650,macOS,1.37,1633.628736
...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4,128.0,Intel HD Graphics 520,Windows 10,1.80,577.874880
1299,Lenovo,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16,512.0,Intel HD Graphics 520,Windows 10,1.30,1357.734240
1300,Lenovo,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2,64.0,Intel HD Graphics,Windows 10,1.50,207.419040
1301,HP,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6,1.0,AMD Radeon R5 M330,Windows 10,2.19,692.000640


### Preliminary Data Analysis

(DESCRIPTION)

REMOVE IF UNNECESSARY

### Separation Into Train and Test Data

We separate the data into train data (which will be used to develop the model), and test data (which will be used to gauge the accuracy of the model). The test data will not be seen by the model until it comes the time to test how well it performs.

In [7]:
laptop_train, laptop_test = train_test_split(
    laptop_data,
    test_size=.25,   # Test data will be a quarter of the full data set, train the rest
)
X_train = laptop_train[["ScreenSize_Inches", "Memory_GB", "Storage", "Weight_Kg"]]
y_train = laptop_train["Price_CAD"]

X_test = laptop_test[["ScreenSize_Inches", "Memory_GB", "Storage", "Weight_Kg"]]
y_test = laptop_test["Price_CAD"]


In [8]:
laptop_train.head(5)

,Company,TypeName,ScreenSize_Inches,ScreenResolution,Cpu,Memory_GB,Storage,Gpu,OpSys,Weight_Kg,Price_CAD
241,Asus,Notebook,17.3,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,128.0,Nvidia GeForce 150MX,Windows 10,2.10,1037.09520
1080,Lenovo,Ultrabook,12.5,IPS Panel Touchscreen 2560x1440,Intel Core M 6Y75 1.2GHz,8,512.0,Intel HD Graphics 515,Windows 10,0.99,1267.15824
147,Asus,Notebook,15.6,Full HD 1920x1080,Intel Celeron Dual Core N3350 1.1GHz,4,1.0,Intel HD Graphics 500,Windows 10,2.00,311.58144
746,Samsung,Ultrabook,13.3,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16,256.0,Intel HD Graphics 620,Windows 10,0.81,1493.59824
1270,Lenovo,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4,128.0,Intel HD Graphics 520,Windows 10,1.80,577.87488


## Building the Model

To select the optimal value for k in k-nearest neighbors (k-NN) regression, we will utilize cross-validation on our training data. We will employ the Root Mean Squared Percentage Error (RMSPE) as the scoring metric. A smaller RMSPE indicates that the predicted values closely align with the true values, while a larger RMSPE suggests a larger deviation.

First, we will create a pipeline for k-NN regression. The pipeline will consist of the k-NN algorithm, along with the StandardScaler for standardizing the numerical features. We will store this pipeline in an object called "laptop_pipe". Next, we will perform cross-validation using the cross_validate function with 10 folds, specifying that we will use the negative RMSPE ("neg_root_mean_squared_error") as the scoring metric.

In [9]:
knn=KNeighborsRegressor()

laptop_pipe = make_pipeline(StandardScaler(),knn)

laptop_cv=pd.DataFrame(
    cross_validate(
        estimator=laptop_pipe,
        cv=10,
        X=X_train,
        y=y_train,
        scoring="neg_root_mean_squared_error",
        return_train_score=True,
    )
)
laptop_cv

,fit_time,score_time,test_score,train_score
0,0.009125,0.005527,-305.234685,-319.329553
1,0.006921,0.004225,-424.038323,-307.334961
2,0.006394,0.004420,-297.605291,-324.645483
3,0.021984,0.004567,-316.234790,-324.403210
4,0.006535,0.004554,-371.836831,-312.372239
5,0.006319,0.004294,-389.959378,-319.459551
6,0.006747,0.004061,-359.902260,-315.207774
7,0.008230,0.005441,-478.813340,-304.423813
8,0.007900,0.005216,-486.351961,-302.368197
9,0.008137,0.004396,-412.285067,-316.294989


We will now use a `param_grid` to test out 200 values of `k`, in order to find the most effective one to use for our model. We will employ the function `GridSearchCV`, passing the parameters `laptop_pipe`, `cv=10`, `n_jobs=-1` and once again `scoring="neg_root_mean_squared_error`.


In [10]:
param_grid = {"kneighborsregressor__n_neighbors": range(1, 201, 1)}
laptop_tuned = GridSearchCV(laptop_pipe, param_grid, cv=10, n_jobs=-1, scoring="neg_root_mean_squared_error")
laptop_results = pd.DataFrame(laptop_tuned.fit(X_train,y_train).cv_results_).rename(columns={"param_kneighborsregressor__n_neighbors":"n_neighbors"})
laptop_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009585,0.006235,0.005019,0.000648,1,{'kneighborsregressor__n_neighbors': 1},-366.354006,-566.207155,-507.912990,-352.207890,-439.688967,-450.104452,-525.761779,-513.396457,-425.166230,-441.336595,-458.813652,65.725352,163
1,0.007434,0.001240,0.004556,0.000688,2,{'kneighborsregressor__n_neighbors': 2},-350.356481,-479.366096,-385.325331,-306.255345,-379.655262,-440.209775,-438.557007,-483.435669,-438.677215,-416.819978,-411.865816,53.494498,61
2,0.007955,0.001242,0.006214,0.003241,3,{'kneighborsregressor__n_neighbors': 3},-324.246478,-432.288571,-344.998681,-320.434125,-381.694261,-398.284843,-396.126447,-496.651245,-427.545658,-428.209854,-395.048016,52.096899,31
3,0.006487,0.000713,0.004116,0.000199,4,{'kneighborsregressor__n_neighbors': 4},-303.991217,-432.753207,-317.692089,-318.204644,-376.327705,-391.261737,-366.693030,-472.877580,-468.583928,-409.823710,-385.820885,57.967706,21
4,0.006107,0.000097,0.004094,0.000109,5,{'kneighborsregressor__n_neighbors': 5},-305.234685,-424.038323,-297.605291,-316.234790,-371.836831,-389.959378,-359.902260,-478.813340,-486.351961,-412.285067,-384.226193,63.947163,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.006271,0.000232,0.008126,0.000282,196,{'kneighborsregressor__n_neighbors': 196},-420.309646,-476.313316,-378.800214,-408.105427,-396.803984,-528.770869,-461.207380,-628.747243,-517.841209,-545.647841,-476.254713,75.078066,196
196,0.006209,0.000149,0.008044,0.000098,197,{'kneighborsregressor__n_neighbors': 197},-420.533739,-476.477278,-379.367136,-408.458135,-397.539402,-529.140771,-461.879941,-628.966665,-518.471815,-546.117043,-476.695192,75.013434,197
197,0.006299,0.000209,0.008352,0.000387,198,{'kneighborsregressor__n_neighbors': 198},-420.967921,-476.897204,-379.652979,-408.904761,-397.797293,-529.748057,-462.811444,-629.786912,-518.740828,-546.455889,-477.176329,75.113046,198
198,0.006873,0.000961,0.008999,0.000964,199,{'kneighborsregressor__n_neighbors': 199},-421.380414,-477.399263,-380.602024,-408.904439,-397.819917,-530.174470,-463.286837,-630.516601,-519.255981,-547.328853,-477.666880,75.234931,199


We can visualize this to get an idea of how the RMSE changes with the number of neighbors.

In [11]:
knn_graph = alt.Chart(laptop_results, title="Neg Mean RMSE vs K from Grid Search Algorithm").mark_line().encode(
    x=alt.X("n_neighbors").title("Number of neighbors"),
    y=alt.Y("mean_test_score")
        .title("Negative mean RMSE")
        .scale(zero=False)
)
knn_graph

alt.Chart(...)

As this is negative RMSE, the highest value on the y-axis will be the most ideal. The RMSE appears to rise sharply up until around `K=10` before decreasing at a constant rate. So now, let's use python to determine the best parameter for `K` as well as the lowest RMSE produced.

In [12]:
laptop_param = laptop_tuned.best_params_
laptop_RMSE = -laptop_tuned.best_score_

laptop_param

{'kneighborsregressor__n_neighbors': 11}

In [13]:
laptop_RMSE

376.8411474729222

## Putting the Model to the Test

To evaluate the performance of our model in predicting unseen data, we will calculate the Root Mean Squared Predicted Error (RMSPE) on the test data. We will utilize the predict function to generate predictions on the test data using `laptop_tuned` (which automatically uses the best `k` parameter) and store the results in a variable called "laptop_prediction". Then, we will compute the RMSPE on the test data using the mean_squared_error function.

In [19]:
laptop_prediction = laptop_tuned.predict(X_test)

laptop_prediction_vs_real = laptop_test.assign(Predicted_CAD = laptop_prediction)
laptop_prediction_vs_real

,Company,TypeName,ScreenSize_Inches,ScreenResolution,Cpu,Memory_GB,Storage,Gpu,OpSys,Weight_Kg,Price_CAD,Predicted_CAD
355,Dell,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,128.0,AMD Radeon 530,Windows 10,2.02,879.402384,751.183822
69,Asus,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,12,1.0,Nvidia GeForce GTX 1050 Ti,Linux,3.00,859.566240,949.648189
202,Acer,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,1.0,Nvidia GeForce 940MX,Windows 10,2.23,672.073920,618.799587
678,LG,Ultrabook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8,512.0,Intel HD Graphics 620,Windows 10,1.09,2082.342240,1508.544103
71,Dell,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,64,256.0,AMD Radeon 530,Windows 10,1.40,865.000800,2952.929109
...,...,...,...,...,...,...,...,...,...,...,...,...
722,Lenovo,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256.0,Intel HD Graphics 620,Windows 10,1.32,1357.734240,1367.285891
635,Asus,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8,1.0,Nvidia GeForce GTX 1050,Windows 10,1.99,821.180131,641.278080
21,Lenovo,Gaming,15.6,IPS Panel Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8,128.0,Nvidia GeForce GTX 1050,Windows 10,2.50,904.854240,963.465146
461,Acer,Netbook,11.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4,128.0,Intel HD Graphics 400,Windows 10,1.40,394.005600,464.405384


Now we find the RMSPE of this prediction.

In [18]:
summary = mean_squared_error(laptop_prediction, y_test)**(1/2)

summary

386.2866224507617